# Network Flow for Assignment Problems

This notebook demonstrates network flow techniques for solving assignment problems in tracking applications. We cover:

1. **Assignment Problem Basics** - Matching measurements to tracks
2. **Network Flow Formulation** - Converting assignments to flow networks
3. **Successive Shortest Paths** - Finding optimal flow
4. **Dijkstra-Optimized Algorithm** - High-performance implementation
5. **Comparison with Hungarian Algorithm** - Performance trade-offs

## Prerequisites

```bash
pip install nrl-tracker matplotlib numpy scipy
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from matplotlib.patches import ArrowStyle
import networkx as nx
import time

from pytcl.assignment_algorithms.network_flow import (
    FlowEdge, FlowStatus,
    assignment_to_flow_network,
    min_cost_flow_successive_shortest_paths,
    min_cost_flow_simplex,
    assignment_from_flow_solution,
    min_cost_assignment_via_flow,
)
from pytcl.assignment_algorithms import hungarian_assignment

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 1. The Assignment Problem in Tracking

In multi-target tracking, each time step requires associating:
- **Tracks**: Existing estimates of target states
- **Measurements**: New sensor observations

The assignment problem finds the optimal one-to-one matching that minimizes total cost.

### Problem Formulation

Given:
- $m$ tracks and $n$ measurements
- Cost matrix $C$ where $C_{ij}$ is cost of assigning track $i$ to measurement $j$

Find: Assignment matrix $X$ where $X_{ij} \in \{0, 1\}$ minimizing:

$$\min \sum_{i,j} C_{ij} X_{ij}$$

Subject to:
$$\sum_j X_{ij} \leq 1 \quad \forall i \quad \text{(each track assigned at most once)}$$
$$\sum_i X_{ij} \leq 1 \quad \forall j \quad \text{(each measurement assigned at most once)}$$

In [ ]:
# Example: 3 tracks, 4 measurements
# Cost based on distance/likelihood

# Track positions
tracks = np.array([
    [0, 0],    # Track 0 at origin
    [5, 0],    # Track 1 at (5, 0)
    [2.5, 4],  # Track 2 at (2.5, 4)
])

# Measurement positions
measurements = np.array([
    [0.1, 0.2],   # Measurement 0 near Track 0
    [4.8, -0.1],  # Measurement 1 near Track 1
    [2.6, 4.1],   # Measurement 2 near Track 2
    [10, 10],     # Measurement 3 (spurious/clutter)
])

# Compute cost matrix (squared distance)
n_tracks = len(tracks)
n_meas = len(measurements)

cost_matrix = np.zeros((n_tracks, n_meas))
for i in range(n_tracks):
    for j in range(n_meas):
        cost_matrix[i, j] = np.sum((tracks[i] - measurements[j])**2)

print("Cost Matrix (squared distance):")
print(f"{'':10s}", end='')
for j in range(n_meas):
    print(f"Meas {j:2d}  ", end='')
print()

for i in range(n_tracks):
    print(f"Track {i:2d}  ", end='')
    for j in range(n_meas):
        print(f"{cost_matrix[i, j]:7.2f}  ", end='')
    print()

In [ ]:
# Visualize tracks and measurements
fig, ax = plt.subplots(figsize=(10, 8))

# Plot tracks
ax.scatter(tracks[:, 0], tracks[:, 1], s=200, c='blue', marker='s', 
           label='Tracks', zorder=5)
for i, (x, y) in enumerate(tracks):
    ax.annotate(f'T{i}', (x, y), xytext=(10, 10), textcoords='offset points', 
                fontsize=12, color='blue')

# Plot measurements
ax.scatter(measurements[:, 0], measurements[:, 1], s=200, c='red', marker='o',
           label='Measurements', zorder=5)
for j, (x, y) in enumerate(measurements):
    ax.annotate(f'M{j}', (x, y), xytext=(10, -15), textcoords='offset points',
                fontsize=12, color='red')

# Draw all possible associations (faint lines)
for i in range(n_tracks):
    for j in range(n_meas):
        ax.plot([tracks[i, 0], measurements[j, 0]], 
                [tracks[i, 1], measurements[j, 1]],
                'gray', alpha=0.1, linewidth=1)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Track-to-Measurement Assignment Problem')
ax.legend()
ax.grid(True)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## 2. Network Flow Formulation

The assignment problem can be converted to a min-cost max-flow network:

```
              c[0,0]          c[0,1]          c[0,2]          c[0,3]
     Worker 0 ─────> Task 0    ──> Task 1    ──> Task 2    ──> Task 3
    /         \                                            /
   /           c[1,0]         c[1,1]         c[1,2]       / c[1,3]
Source ─> W1 ─────────────────────────────────────────────
   \           c[2,0]         c[2,1]         c[2,2]         c[2,3]
    \         Worker 2 ────────────────────────────────────> Sink
```

### Network Structure

- **Source**: Supplies $m$ units of flow (one per track)
- **Worker nodes**: One per track, connected to source with capacity 1
- **Task nodes**: One per measurement, connected to all workers with capacity 1
- **Sink**: Demands $m$ units of flow
- **Edge costs**: Assignment costs $C_{ij}$

In [ ]:
# Convert to flow network
edges, supplies, node_names = assignment_to_flow_network(cost_matrix)

print("Network Structure")
print("=" * 60)
print(f"Number of nodes: {len(supplies)}")
print(f"Number of edges: {len(edges)}")
print(f"\nNodes: {list(node_names)}")

print(f"\nSupplies/Demands:")
for i, (name, supply) in enumerate(zip(node_names, supplies)):
    if supply > 0:
        print(f"  {name}: supplies {supply:.0f}")
    elif supply < 0:
        print(f"  {name}: demands {-supply:.0f}")

print(f"\nEdges with costs:")
for edge in edges[:8]:  # Show first 8 edges
    from_name = node_names[edge.from_node]
    to_name = node_names[edge.to_node]
    print(f"  {from_name} -> {to_name}: capacity={edge.capacity:.0f}, cost={edge.cost:.2f}")
print(f"  ... ({len(edges) - 8} more edges)")

In [ ]:
# Visualize the flow network
fig, ax = plt.subplots(figsize=(14, 8))

# Node positions
pos = {}
pos['source'] = (-2, 2)
pos['sink'] = (8, 2)

# Worker nodes on left
for i in range(n_tracks):
    pos[f'worker_{i}'] = (0, 4 - i * 2)

# Task nodes on right
for j in range(n_meas):
    pos[f'task_{j}'] = (6, 4.5 - j * 1.5)

# Draw nodes
node_colors = {'source': 'green', 'sink': 'orange'}
for name in node_names:
    x, y = pos[name]
    color = node_colors.get(name, 'lightblue' if 'worker' in name else 'lightcoral')
    circle = plt.Circle((x, y), 0.3, color=color, ec='black', linewidth=2, zorder=5)
    ax.add_patch(circle)
    ax.text(x, y, name.replace('worker_', 'W').replace('task_', 'T'), 
            ha='center', va='center', fontsize=9, fontweight='bold')

# Draw edges
for edge in edges:
    from_name = node_names[edge.from_node]
    to_name = node_names[edge.to_node]
    x1, y1 = pos[from_name]
    x2, y2 = pos[to_name]
    
    # Adjust for node radius
    dx, dy = x2 - x1, y2 - y1
    length = np.sqrt(dx**2 + dy**2)
    if length > 0:
        dx, dy = dx/length, dy/length
        x1, y1 = x1 + 0.3*dx, y1 + 0.3*dy
        x2, y2 = x2 - 0.3*dx, y2 - 0.3*dy
    
    arrow = FancyArrowPatch((x1, y1), (x2, y2),
                           arrowstyle='->', mutation_scale=15,
                           color='gray', alpha=0.6)
    ax.add_patch(arrow)
    
    # Add cost label for worker->task edges
    if 'worker' in from_name and 'task' in to_name:
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x, mid_y, f'{edge.cost:.1f}', fontsize=7, 
                ha='center', va='center', color='blue')

ax.set_xlim(-3, 10)
ax.set_ylim(-1, 6)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Min-Cost Flow Network for Assignment', fontsize=14)

# Legend
ax.text(-2.5, 5, 'Supply=3', fontsize=10, color='green')
ax.text(8, 5, 'Demand=3', fontsize=10, color='orange')

plt.tight_layout()
plt.show()

## 3. Successive Shortest Paths Algorithm

The successive shortest paths algorithm finds the minimum cost flow:

### Algorithm
```
while supply nodes have remaining supply:
    1. Find shortest path from supply to demand node
    2. Compute maximum flow that can be pushed
    3. Push flow and update residual capacities
    4. Update node supplies/demands
```

### Complexity
- **Bellman-Ford**: $O(k \cdot V \cdot E)$ where $k$ = number of paths
- **Dijkstra with potentials**: $O(k \cdot E \log V)$

In [ ]:
# Solve using successive shortest paths
result_bf = min_cost_flow_successive_shortest_paths(edges, supplies)

print("Bellman-Ford Based Solution")
print("=" * 50)
print(f"Status: {result_bf.status.name}")
print(f"Total Cost: {result_bf.cost:.2f}")
print(f"Iterations: {result_bf.iterations}")

# Extract assignment
assignment_bf, cost_bf = assignment_from_flow_solution(
    result_bf.flow, edges, cost_matrix.shape
)

print(f"\nOptimal Assignment:")
for row_idx, col_idx in assignment_bf:
    print(f"  Track {row_idx} -> Measurement {col_idx} (cost: {cost_matrix[row_idx, col_idx]:.2f})")

In [ ]:
# Solve using Dijkstra-optimized algorithm
result_dijkstra = min_cost_flow_simplex(edges, supplies)

print("Dijkstra-Optimized Solution")
print("=" * 50)
print(f"Status: {result_dijkstra.status.name}")
print(f"Total Cost: {result_dijkstra.cost:.2f}")
print(f"Iterations: {result_dijkstra.iterations}")

# Extract assignment
assignment_dij, cost_dij = assignment_from_flow_solution(
    result_dijkstra.flow, edges, cost_matrix.shape
)

print(f"\nOptimal Assignment:")
for row_idx, col_idx in assignment_dij:
    print(f"  Track {row_idx} -> Measurement {col_idx} (cost: {cost_matrix[row_idx, col_idx]:.2f})")

In [ ]:
# Visualize the optimal assignment
fig, ax = plt.subplots(figsize=(10, 8))

# Plot tracks
ax.scatter(tracks[:, 0], tracks[:, 1], s=200, c='blue', marker='s', 
           label='Tracks', zorder=5)
for i, (x, y) in enumerate(tracks):
    ax.annotate(f'T{i}', (x, y), xytext=(10, 10), textcoords='offset points', 
                fontsize=12, color='blue')

# Plot measurements
ax.scatter(measurements[:, 0], measurements[:, 1], s=200, c='red', marker='o',
           label='Measurements', zorder=5)
for j, (x, y) in enumerate(measurements):
    ax.annotate(f'M{j}', (x, y), xytext=(10, -15), textcoords='offset points',
                fontsize=12, color='red')

# Draw optimal assignments with thick green lines
for row_idx, col_idx in assignment_dij:
    ax.plot([tracks[row_idx, 0], measurements[col_idx, 0]], 
            [tracks[row_idx, 1], measurements[col_idx, 1]],
            'g-', linewidth=3, alpha=0.8, label='Assignment' if row_idx == assignment_dij[0, 0] else '')

# Highlight unassigned measurement
assigned_meas = set(assignment_dij[:, 1])
for j in range(n_meas):
    if j not in assigned_meas:
        ax.scatter(measurements[j, 0], measurements[j, 1], s=300, 
                   facecolors='none', edgecolors='purple', linewidth=2,
                   label='Unassigned')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title(f'Optimal Assignment (Total Cost: {cost_dij:.2f})')
ax.legend()
ax.grid(True)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## 4. Performance Comparison

Let's compare the performance of different algorithms:
1. Network flow (Bellman-Ford)
2. Network flow (Dijkstra-optimized)
3. Hungarian algorithm

In [ ]:
# Performance benchmark
sizes = [10, 20, 50, 100, 200]

bf_times = []
dijkstra_times = []
hungarian_times = []

n_trials = 5

for size in sizes:
    # Generate random cost matrix
    cost = np.random.rand(size, size) * 100
    
    # Bellman-Ford based (limit to smaller sizes)
    if size <= 50:
        edges_tmp, supplies_tmp, _ = assignment_to_flow_network(cost)
        times = []
        for _ in range(n_trials):
            start = time.time()
            result = min_cost_flow_successive_shortest_paths(edges_tmp, supplies_tmp)
            times.append(time.time() - start)
        bf_times.append(np.mean(times))
    else:
        bf_times.append(np.nan)
    
    # Dijkstra-optimized
    edges_tmp, supplies_tmp, _ = assignment_to_flow_network(cost)
    times = []
    for _ in range(n_trials):
        start = time.time()
        result = min_cost_flow_simplex(edges_tmp, supplies_tmp)
        times.append(time.time() - start)
    dijkstra_times.append(np.mean(times))
    
    # Hungarian algorithm
    times = []
    for _ in range(n_trials):
        start = time.time()
        row_ind, col_ind = hungarian_assignment(cost)
        times.append(time.time() - start)
    hungarian_times.append(np.mean(times))
    
    print(f"Size {size:3d}x{size:3d}: BF={bf_times[-1]*1e3:7.2f}ms, "
          f"Dijkstra={dijkstra_times[-1]*1e3:7.2f}ms, "
          f"Hungarian={hungarian_times[-1]*1e3:7.2f}ms")

In [ ]:
# Visualize performance comparison
fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogy(sizes, np.array(bf_times)*1e3, 'r-o', label='Bellman-Ford', linewidth=2)
ax.semilogy(sizes, np.array(dijkstra_times)*1e3, 'g-s', label='Dijkstra-optimized', linewidth=2)
ax.semilogy(sizes, np.array(hungarian_times)*1e3, 'b-^', label='Hungarian', linewidth=2)

ax.set_xlabel('Problem Size (n x n)')
ax.set_ylabel('Time (ms)')
ax.set_title('Assignment Algorithm Performance Comparison')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

## 5. Application: Multi-Target Tracking

Let's demonstrate using network flow for a complete tracking scenario with:
- Gate-based cost computation
- Missed detection handling
- New track initiation

In [ ]:
# Simulate tracking scenario
n_timesteps = 10
n_tracks = 5

# True target trajectories
np.random.seed(42)
true_positions = []
positions = np.random.randn(n_tracks, 2) * 5 + 10
velocities = np.random.randn(n_tracks, 2) * 0.5

for t in range(n_timesteps):
    true_positions.append(positions.copy())
    positions += velocities + np.random.randn(n_tracks, 2) * 0.1

# Generate measurements with noise and clutter
detection_prob = 0.9
clutter_rate = 2  # average clutter per frame
measurement_noise = 0.3

all_measurements = []
all_truth_associations = []

for t in range(n_timesteps):
    frame_meas = []
    frame_assoc = []
    
    # Target detections
    for i in range(n_tracks):
        if np.random.rand() < detection_prob:
            meas = true_positions[t][i] + np.random.randn(2) * measurement_noise
            frame_meas.append(meas)
            frame_assoc.append(i)
    
    # Clutter
    n_clutter = np.random.poisson(clutter_rate)
    for _ in range(n_clutter):
        clutter = np.random.rand(2) * 30  # Uniform in surveillance region
        frame_meas.append(clutter)
        frame_assoc.append(-1)  # -1 indicates clutter
    
    all_measurements.append(np.array(frame_meas))
    all_truth_associations.append(np.array(frame_assoc))

print(f"Generated {n_timesteps} frames with {n_tracks} targets")
for t in range(3):
    n_det = np.sum(all_truth_associations[t] >= 0)
    n_clut = np.sum(all_truth_associations[t] < 0)
    print(f"  Frame {t}: {n_det} detections, {n_clut} clutter")

In [ ]:
def compute_gated_cost_matrix(track_positions, measurements, gate_threshold=10.0):
    """
    Compute cost matrix with gating.
    
    Parameters
    ----------
    track_positions : ndarray
        Track predicted positions [n_tracks, 2].
    measurements : ndarray
        Measurement positions [n_meas, 2].
    gate_threshold : float
        Maximum squared distance for valid association.
        
    Returns
    -------
    cost_matrix : ndarray
        Cost matrix with inf for gated-out associations.
    """
    n_tracks = len(track_positions)
    n_meas = len(measurements)
    
    cost_matrix = np.full((n_tracks, n_meas), np.inf)
    
    for i in range(n_tracks):
        for j in range(n_meas):
            dist_sq = np.sum((track_positions[i] - measurements[j])**2)
            if dist_sq < gate_threshold:
                cost_matrix[i, j] = dist_sq
    
    return cost_matrix


# Run tracking with network flow assignment
# Initialize tracks at frame 0
track_estimates = true_positions[0].copy()

assignment_results = []
total_correct = 0
total_assignments = 0

for t in range(1, n_timesteps):
    meas = all_measurements[t]
    truth_assoc = all_truth_associations[t]
    
    if len(meas) == 0:
        continue
    
    # Compute gated cost matrix
    cost = compute_gated_cost_matrix(track_estimates, meas, gate_threshold=4.0)
    
    # Replace inf with large value for network flow
    max_cost = 1e6
    cost_finite = np.where(np.isinf(cost), max_cost, cost)
    
    # Solve assignment
    assignment, total_cost = min_cost_assignment_via_flow(cost_finite)
    
    # Evaluate accuracy
    for row_idx, col_idx in assignment:
        if cost[row_idx, col_idx] < max_cost:  # Valid assignment
            total_assignments += 1
            if truth_assoc[col_idx] == row_idx:  # Correct assignment
                total_correct += 1
    
    # Update track estimates with assigned measurements
    for row_idx, col_idx in assignment:
        if cost[row_idx, col_idx] < max_cost:
            track_estimates[row_idx] = meas[col_idx]
    
    assignment_results.append((t, assignment, cost))

accuracy = total_correct / total_assignments if total_assignments > 0 else 0
print(f"\nTracking Results:")
print(f"  Total assignments: {total_assignments}")
print(f"  Correct associations: {total_correct}")
print(f"  Accuracy: {accuracy*100:.1f}%")

In [ ]:
# Visualize tracking results
fig, ax = plt.subplots(figsize=(12, 10))

colors = plt.cm.tab10(np.linspace(0, 1, n_tracks))

# Plot true trajectories
for i in range(n_tracks):
    traj = np.array([true_positions[t][i] for t in range(n_timesteps)])
    ax.plot(traj[:, 0], traj[:, 1], '-', color=colors[i], linewidth=2, 
            label=f'Target {i}' if i < 3 else '', alpha=0.7)
    ax.scatter(traj[0, 0], traj[0, 1], s=100, c=[colors[i]], marker='o', zorder=5)
    ax.scatter(traj[-1, 0], traj[-1, 1], s=100, c=[colors[i]], marker='s', zorder=5)

# Plot measurements
for t in range(n_timesteps):
    meas = all_measurements[t]
    truth = all_truth_associations[t]
    
    for j in range(len(meas)):
        if truth[j] >= 0:
            # Detection
            ax.scatter(meas[j, 0], meas[j, 1], s=20, c=[colors[truth[j]]], 
                       marker='.', alpha=0.5)
        else:
            # Clutter
            ax.scatter(meas[j, 0], meas[j, 1], s=20, c='gray', 
                       marker='x', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title(f'Multi-Target Tracking with Network Flow Assignment\n'
             f'(Accuracy: {accuracy*100:.1f}%)')
ax.legend(loc='upper left')
ax.grid(True)

plt.tight_layout()
plt.show()

## Summary

Key takeaways:

1. **Network flow formulation** converts assignment to min-cost flow
2. **Successive shortest paths** finds optimal solution iteratively
3. **Dijkstra optimization** provides O(k*E log V) vs O(k*V*E)
4. **Hungarian algorithm** is often faster for dense problems
5. **Network flow** scales better for sparse/gated problems

### When to Use Each Algorithm

| Scenario | Recommended Algorithm |
|----------|----------------------|
| Small dense problems (n < 100) | Hungarian |
| Large dense problems | Dijkstra-optimized flow |
| Sparse/gated problems | Network flow |
| Rectangular problems | Network flow |

## Exercises

1. Implement a custom cost function for track-measurement association
2. Add dummy nodes to handle missed detections and new tracks
3. Compare network flow with auction algorithm for large problems
4. Implement 3D assignment using min-cost flow relaxation

## References

1. Ahuja, R. K., Magnanti, T. L., & Orlin, J. B. (1993). *Network Flows*.
2. Blackman, S., & Popoli, R. (1999). *Design and Analysis of Modern Tracking Systems*.
3. Bar-Shalom, Y., & Li, X. R. (1995). *Multitarget-Multisensor Tracking*.